In [ ]:
!pip install underthesea


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 54.7 MB/s eta 0:00:00


In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

from underthesea import word_tokenize

from keras import Input, Model
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
import math
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('chatbot3.csv', usecols=[0,1])
df

,question,answers
0,Làm thế nào mà thí nghiệm thành công?,chúng vẫn ấm và khô qua mùa đông
1,Khi nào phiên tiếp theo được công bố?,12 tháng 11 năm 1962
2,"Ai nói rằng Chopin đặt ra ""vào thế giới rộng l...",Zdzisław Jachimecki
3,Ai là quan chức cấp cao nhất trong Chính phủ N...,Thống đốc Chiết Giang
4,Oklahoma có bao nhiêu trường đại học công lập?,mười một
...,...,...
9993,Một nghiên cứu của ProPublica cho thấy một số ...,hơn 500.000 đô la
9994,Tên của con tàu đầu tiên sử dụng đèn của Ediso...,Columbia
9995,Ở những nước nào quặng uranium cao cấp đáng ch...,Canada
9996,Những trang trại sử dụng thuốc trừ sâu sinh họ...,trang trại độc canh


In [ ]:
data_questions = df['question'].values
data_answers = df['answers'].values

In [ ]:
#XỬ lý dữ liệu
def clean_text(sent):
    return re.sub(r'[!“”"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]', '', sent)

# hàm để chuyển Word Segmentation cho tiếng Việt
def clean_and_word_segmentation(sent):
    # Handle None or NaN values
    if sent is None or (isinstance(sent, float) and math.isnan(sent)):
        return ''
    # Convert to string
    sent = str(sent)
    # Check for rare data
    if len(sent) < 3:  # Adjust this threshold based on your data characteristics
        return ''
    # Clean and tokenize the text
    return word_tokenize(clean_text(sent.lower()))

In [ ]:
count_words_ques = [len(clean_text(ques).split()) for ques in data_questions]
counter_words_ques = Counter(count_words_ques)

list_count_word = []
list_count_sent = []
for i in counter_words_ques.items():
    #print(i)
    list_count_word.append(i[0])
    list_count_sent.append(i[1])


In [ ]:
len(count_words_ques), len(data_questions)


(9998, 9998)

In [ ]:

sorted_ques = []
sorted_ans = []
for i,count in enumerate(count_words_ques):
    if count <= 15:
        sorted_ques.append(data_questions[i])
        sorted_ans.append(data_answers[i])

print('len sorted_ques:', len(sorted_ques))
print('len sorted_ans:', len(sorted_ans))
sorted_ques[:2], sorted_ans[:2]

len sorted_ques: 7182
len sorted_ans: 7182


(['Làm thế nào mà thí nghiệm thành công?',
  'Khi nào phiên tiếp theo được công bố?'],
 ['chúng vẫn ấm và khô qua mùa đông', '12 tháng 11 năm 1962'])

In [ ]:
# Clean and word segmentation for sorted_ques and sorted_ans
questions = [' '.join(clean_and_word_segmentation(ques)) for ques in sorted_ques]
answers = ['<START> ' + ' '.join(clean_and_word_segmentation(answ)) + ' <END>' for answ in sorted_ans]


print(questions[:2])
print(answers[:2])


['làm thế nào mà thí nghiệm thành công', 'khi nào phiên tiếp theo được công bố']
['<START> chúng vẫn ấm và khô qua mùa đông <END>', '<START> 12 tháng 11 năm 1962 <END>']


In [ ]:
# tokenize cho questions và answers
tokenizer = Tokenizer(filters='', lower=False)
tokenizer.fit_on_texts(questions + answers)

In [ ]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
print(f'Vocabulary size : {VOCAB_SIZE}')

Vocabulary size : 8829


In [ ]:
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = 15
encoder_inp = pad_sequences(tokenized_questions,maxlen=maxlen_questions,padding='post')

print(encoder_inp.shape)
print(questions[0])
print(tokenized_questions[0])
print(encoder_inp[0])

(7182, 15)
làm thế nào mà thí nghiệm thành công
[26, 28, 3, 147, 1007, 461, 25, 27]
[  26   28    3  147 1007  461   25   27    0    0    0    0    0    0
    0]


In [ ]:
# decoder
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = np.max([len(x) for x in tokenized_answers])
decoder_inp = pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')

print(decoder_inp.shape)
print(answers[0])
print(tokenized_answers[0])
print(decoder_inp[0])

(7182, 45)
<START> chúng vẫn ấm và khô qua mùa đông <END>
[1, 533, 594, 2099, 13, 1472, 245, 211, 226, 2]
[   1  533  594 2099   13 1472  245  211  226    2    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]


In [ ]:
for i in range(len(tokenized_answers)):
    tokenized_answers[i] = tokenized_answers[i][1:]

padded_answers = pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
decoder_final_output = to_categorical(padded_answers, VOCAB_SIZE)

print(decoder_final_output.shape)
print(tokenized_answers[0])
print(padded_answers[0])
print(decoder_final_output[0])

(7182, 45, 8829)
[533, 594, 2099, 13, 1472, 245, 211, 226, 2]
[ 533  594 2099   13 1472  245  211  226    2    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [ ]:
enc_inputs = Input(shape=(None,))
enc_embedding = Embedding(VOCAB_SIZE, 200, mask_zero=True)(enc_inputs)
_, state_h, state_c = LSTM(200, return_state=True)(enc_embedding)
enc_states = [state_h, state_c]

dec_inputs = Input(shape=(None,))
dec_embedding = Embedding(VOCAB_SIZE, 200, mask_zero=True)(dec_inputs)
dec_lstm = LSTM(200, return_state=True, return_sequences=True)

dec_outputs, _, _ = dec_lstm(dec_embedding, initial_state=enc_states)
dec_dense = Dense(VOCAB_SIZE, activation='softmax')
output = dec_dense(dec_outputs)

In [ ]:
model = Model([enc_inputs, dec_inputs], output)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 200)            1765800   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 200)            1765800   ['input_2[0][0]']             
                                                                                              

In [ ]:
def data_generator(encoder_inp, decoder_inp, decoder_final_output, batch_size):
    num_samples = len(encoder_inp)
    while True:
        for offset in range(0, num_samples, batch_size):
            encoder_batch = encoder_inp[offset:offset+batch_size]
            decoder_input_batch = decoder_inp[offset:offset+batch_size]
            decoder_output_batch = decoder_final_output[offset:offset+batch_size]
            yield [encoder_batch, decoder_input_batch], decoder_output_batch

batch_size = 32  # Set an appropriate batch size
model.fit(data_generator(encoder_inp, decoder_inp, decoder_final_output, batch_size),
          steps_per_epoch=len(encoder_inp) // batch_size,
          epochs=200)

model.save('chatbot.h5')

Epoch 1/200
224/224 [==============================] - 16s 72ms/step - loss: 2.0273
Epoch 2/200
224/224 [==============================] - 17s 76ms/step - loss: 2.0009
Epoch 3/200
224/224 [==============================] - 16s 72ms/step - loss: 1.9733
Epoch 4/200
224/224 [==============================] - 17s 74ms/step - loss: 1.9446
Epoch 5/200
224/224 [==============================] - 16s 72ms/step - loss: 1.9205
Epoch 6/200
224/224 [==============================] - 17s 74ms/step - loss: 1.8953
Epoch 7/200
224/224 [==============================] - 17s 74ms/step - loss: 1.8670
Epoch 8/200
224/224 [==============================] - 16s 72ms/step - loss: 1.8380
Epoch 9/200
224/224 [==============================] - 17s 75ms/step - loss: 1.8153
Epoch 10/200
224/224 [==============================] - 18s 78ms/step - loss: 1.7840
Epoch 11/200
224/224 [==============================] - 16s 73ms/step - loss: 1.7646
Epoch 12/200
224/224 [==============================] - 16s 71ms/step - lo

In [ ]:
def make_inference_models():
    dec_state_input_h = Input(shape=(200,))
    dec_state_input_c = Input(shape=(200,))
    dec_states_inputs = [dec_state_input_h, dec_state_input_c]
    dec_outputs, state_h, state_c = dec_lstm(dec_embedding,
                                            initial_state=dec_states_inputs)
    dec_states = [state_h, state_c]
    dec_outputs = dec_dense(dec_outputs)
    dec_model = Model(
        inputs=[dec_inputs] + dec_states_inputs,
        outputs = [dec_outputs] + dec_states)
    print('Inference decoder:')
    dec_model.summary()
    print('Inference encoder:')
    enc_model = Model(inputs=enc_inputs, outputs=enc_states)
    enc_model.summary()
    return enc_model, dec_model




enc_model, dec_model = make_inference_models()

Inference decoder:
Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, None, 200)            1765800   ['input_2[0][0]']             
                                                                                                  
 input_13 (InputLayer)       [(None, 200)]                0         []                            
                                                                                                  
 input_14 (InputLayer)       [(None, 200)]                0         []                            
                                                                        

In [ ]:
def str_to_tokens(sentence):
    cleaned_and_segmented_sentence = clean_and_word_segmentation(sentence)
    if isinstance(cleaned_and_segmented_sentence, list):
        cleaned_and_segmented_sentence = ' '.join(cleaned_and_segmented_sentence)
    words = cleaned_and_segmented_sentence.lower().split()
    tokens_list = list()
    for current_word in words:
        result = tokenizer.word_index.get(current_word)
        if result is not None:
            tokens_list.append(result)
    return pad_sequences([tokens_list], maxlen=maxlen_questions, padding='post')


In [ ]:
def chatbot():
    print('Bot: Xin chào!')

    while True:
        input_question = input('Question: ')

        if input_question == 'bye':
            print('Bot answer: bye')
            break
        states_values = enc_model.predict(str_to_tokens(input_question))
        empty_target_seq = np.zeros((1,1))
        empty_target_seq[0,0] = tokenizer.word_index['<START>']
        stop_condition = False
        decoded_translation = ''
        while not stop_condition:
            dec_outputs, h, c = dec_model.predict([empty_target_seq]+states_values)
            sampled_word_index = np.argmax(dec_outputs[0,-1, :])
            sampled_word = None
            for word, index in tokenizer.word_index.items():
                if sampled_word_index == index:
                    if word != '<END>':
                        decoded_translation += f'{word} '
                    sampled_word = word

            if sampled_word == '<END>' or len(decoded_translation.split()) > maxlen_answers:
                stop_condition = True
            empty_target_seq = np.zeros((1,1))
            empty_target_seq[0,0] = sampled_word_index
            states_values = [h,c]

        print('Bot answer:', decoded_translation, '\n')

In [ ]:
chatbot()

Bot: Xin chào!
Question: làm thế nào để thí nghiệm
1/1 [==============================] - 0s 19ms/step
Bot answer: trên mực nước biển  

Question: làm thế nào mà thí nghiệm thành công
1/1 [==============================] - 0s 18ms/step
Bot answer: chúng vẫn ấm và khô qua mùa đông  

Question: Khi nào phiên tiếp theo được công bố?
1/1 [==============================] - 0s 18ms/step
Bot answer: 12 tháng 11 năm 1962  

Question: Thẩm phán nào là Chánh án Toà án tối cao Tuvalu?
1/1 [==============================] - 0s 18ms/step
Bot answer: ngài gordon ward  

Question: Oklahoma có bao nhiêu trường đại học công lập?	
1/1 [==============================] - 0s 18ms/step
Bot answer: mười một  

Question: bye
Bot answer: bye
